# CRISPRi for transcription regulation

First import the API and other packages we need:

In [ ]:
import flapjack
from flapjack import Flapjack
import numpy as np
import plotly
import plotly.graph_objects as go
import plotly.io as io
import getpass
import pandas as pd
import seaborn as sns
%matplotlib inline

Next we connect to Flapjack by creating a session with our login:

In [ ]:
user = input()
passwd = getpass.getpass()
fj = Flapjack('flapjack.rudge-lab.org:8000')
fj.log_in(username=user, password=passwd)

## Figure 7B - Induction curves

As well as using the Flapjack webapp (https://github.com/SynBioUC/flapjack_frontend/wiki/dCas9-Inverters), you can obtain the figure 7B using the Flapjack Python package. Filter the data to select: 
* study: "dCas9 Inverters", 
* vector (plasmid): 'CcaT+TMA2','CcaT+TMA4','CcaT+TMA5',

To plot all points and not just the mean, select 'All data points' in plot: 

* normalize='None',
* subplots='Signal',
* markers='Vector',
* plot='All data points'

In [ ]:
study_id = fj.get('study', name='dCas9 Inverters').id
vector_names = ['CcaT+TMA2','CcaT+TMA4','CcaT+TMA5']
vector_ids = [fj.get('vector', name=v).id for v in vector_names]
iptg_id = fj.get('chemical', name='IPTG').id
biomass_signal_id = fj.get('signal', name='OD').id[0]

In [ ]:
fig = fj.plot(study=study_id,                     
                vector=vector_ids,
                type='Induction Curve',
                analyte=iptg_id,
                function='Mean Expression',
                biomass_signal=biomass_signal_id,
                normalize='None',
                subplots='Signal',
                markers='Vector',
                plot='All data points'
       )
fig

## Figure 7C - Fit Hill functions

In [ ]:
params = pd.DataFrame()
for vec in vector_names:
    vector_id = fj.get('vector', name=vec).id
    df = fj.analysis(vector=vector_id,
                     analyte=iptg_id,
                     type='Induction Curve',
                     function='Mean Expression',
                     biomass_signal=biomass_signal_id
                    )
    for sig in ['YFP', 'CFP']:
        data = df[df.Signal==sig]
        p,std = flapjack.fit_curve(
            flapjack.hill, 
            data, 
            x='Concentration', y='Expression',
            bounds=[[0,0,0,0], [1e6,1e6,1e-3,4]]
        )
        params = params.append({
            'Vector': vec, 
            'Signal': sig,
            'a': p[0],
            'b': p[1],
            'K': p[2],
            'n': p[3],
        }, ignore_index=True) 
        #[(vec, sig)] = p, std
    sig = 'OD'
    data = df[df.Signal==sig]
    p,std = flapjack.fit_curve(
        flapjack.hill, 
        data, 
        x='Concentration', y='Expression',
        bounds=[[0,0,0,0], [2,2,1e-3,4]]
    )
    params = params.append({
        'Vector': vec, 
        'Signal': sig,
        'a': p[0],
        'b': p[1],
        'K': p[2],
        'n': p[3],
    }, ignore_index=True) 
    #params[(vec, sig)] = p, std

In [ ]:
params.to_csv('CRISPRi_induction_params.csv')

## Figure 7D - Kymographs

As well as using the Flapjack webapp (https://github.com/SynBioUC/flapjack_frontend/wiki/dCas9-Inverters), you can obtain the figure 7D using the Flapjack Python package.

In [ ]:
study_id = fj.get('study', name='dCas9 Inverters').id
biomass_signal_id = fj.get('signal', name='OD').id[0]
vector_names = ['CcaT+TMA2', 'CcaT+TMA4', 'CcaT+TMA5']
iptg_id = fj.get('chemical', name='IPTG').id[0]
for vec in vector_names:
    vector_id = fj.get('vector', name=vec).id
    fig = fj.plot(study=study_id,
                 vector=vector_id,
                 type='Kymograph',
                 analyte=iptg_id,
                 function='Expression Rate (indirect)',
                 pre_smoothing=11, post_smoothing=11,
                 biomass_signal=biomass_signal_id,
                 normalize='None',
                 subplots='Signal',
                 markers='Vector',
                 plot='Mean')
    fig = flapjack.layout_print(fig, width=4, height=1.5)
    fig.update_xaxes(tickangle=30)
    fname = vec + '_kymograph.png'
    io.write_image(fig, fname)